# Entropy and data compression problems

## Question 1: Biased and unbiased coin flips

### Learning objectives
In this question you will:

- implement compression algorithms
- understand the connection between data compression and entropy


### 1a. Biased coin

First, make a biased-coin simulator that independently outputs heads (say $x = 1$) with probability $\theta$ and tails (say $x = 0$) with probability  $(1-\theta)$ on each "toss", where $0 \le \theta \le 1$ is an input parameter.

In [ ]:
#Write your answer here

### 1b. 

In a suitably large number of tosses (and argue what should be meant by "suitably large"), verify that the mean fraction of heads, standard deviation, and Shannon entropy are approximately as expected theoretically, for 
1. $\theta = 0.125$, 
2. $\theta = 0.48$, and 
3. $\theta = 0.005$.

In [ ]:
#Write your answer here

### 1c. Unbiasing the coin

Suppose we are given access only to biased coin (simulated here by your algorithm introduced above).  If we do _not_ know the probability of heads, but are assured that successive tosses are stochastically independent whatever the marginal probability is, how might we produce random bits independently and with exactly equal probability?

Write your answer here

### 1d. 

You likely came up with the same idea as von Neumann (congratulations!), involving a simple rejection method followed by a mapping of two successive bits into one output bit.

If it turns out that the probability of heads was actually $\theta$, what is the expected number of tosses needed per output bit for such a method? <font color="red">Can the original input be reconstructed from the output?</font>

Write your answer here

### 1e. 

Using your sampler, sample $10^6$ coin flips for each of the previously used values of $\theta$. Then implement the algorithm to obtain un-biased coin flips. How many do we get for each $\theta$? Compare to theoretical expectations. (Also check whether the output bits are unbiased.)

In [ ]:
#Write your answer here

### 1f. Compression

Instead, now suppose that you know the probability of heads, and want to generate uniformly random (equal probability) bits with minimal waste.
How might we produce random bits with equal probability?

One strategy is to use a so-called _entropic compression_ algorithm, which in principle can compress bit streams to within the limits set by information theory.
If we feed in biased bits ($1$ for heads, and $0$ for tails), with less than $1$ bit of entropy per actual bit, into a very good string compression algorithm, the output bits should look fully random (with almost $1$ bit of entropy per output bit).

Given that we are assuming that the bias probability $\theta$ is (somehow) known, a variety of standard compression techniques could be utilized, such as _arithmetic coding_, or what is becoming a favored alternate, a so-called _asymmetric numeral system_, briefly described next.

Suppose $x$ is an integer whose binary representation, from most to least significant bits, is intended to contain the _encoded_, or _compressed_ bit sequence.  If $s \in \{ 0,1\}$ is the next bit to be encoded, we could append it to $x$ by calculating $x' = 2x + s$ and replacing $x \to x'$.  But this would only be efficient in terms of compression ratios if new bits $s$ are arriving with uniform probability ($\theta = \tfrac{1}{2}$), in which case $x$ evolves randomly from even to odd parity, and for large $x$, $x' \approx 2x = \tfrac{x}{\frac{1}{2}}$ after each update.

More generally, we can look for an encoding function that satisfies
$x' = C(x,s) \approx \tfrac{x}{\theta}$.  In other words, rather than splitting integers uniformly into even and odd numbers, we can imagine breaking them up into different sorts of subsets with probabilities given approximately by $\theta$ and $(1-\theta)$.  A simple coding that approximately achieves this is
$$
C(x,s) = \begin{cases}
\left\lceil \tfrac{x+1}{1-\theta} \right\rceil - 1 &\text{ if } s = 0 \\
\left\lfloor \tfrac{x}{\theta} \right\rfloor     &\text{ if } s = 1
\end{cases} .
$$

What is the expected compression ratio $R=\frac{\text{uncompressed size}}{\text{compressed size}}$ for bits with bias $\theta$?

Write your answer here

### 1g. 

To avoid repeated conversion between ints and floats (which is slow), and rounding errors from using floats, we assume that $\theta$ is given as $\theta = \tfrac{a}{l}$, where $a$ and $l$ are integers satisfying $0 
< a < l$. While Python can handle arbitrarily large integers, this is slow, so we'll cap the size of $x$ (and $l$) at $m$ bits, by appending $x$ to the output and re-initialising it to the starting value every time $C(x,s)\geq2^m$, effectively splittingour (compressed) data into $m$-bit blocks.

Using this type of entropic encoding, compress the bits from your biased coin simulator. (What starting value do you need?)

In [ ]:
#Write your answer here

### 1h. 

Compression differs from simply unbiasing coin flips. In unbiasing, we want to extract randomness from less random things, and we want to obtain as many unbiased bits as possible. In compression, we want to reduce the size of data while maintaining bijectivity. No lossless compression algorithm exists that can guarantee to reduce the size of _any_ data. Lossless compression only works with low-entropy data, or when additional information/structure is known about the data (which reduces its entropy). The crucial component of compression is bijectivity (otherwise we could compress anything to 0), so let's test it out.

Show that the inverse $D_s(x')$, $D_x(x')$ of the encoding function $C(x,s)$ is given by $$D_s(x'),\:D_x(x')=\begin{cases}1,\:\lceil\theta x'\rceil&\text{ if } \lceil\theta x'\rceil<\theta(x'+1)&\\0,\:x'-\lceil\theta x'\rceil&\text{ else }\end{cases}.$$

Write your answer here

### 1i. 

Implement the inverse to your compression algorithm, and verify that you can recover the samples you started with.

In [ ]:
#Write your answer here

Congratulations! You have implemented an efficient algorithm for compressing streams of independent bits. More pragmatic compression algorithms might also take advantage of correlations between bits. For an extreme example, if our data distribution is supported only by the bit streams `10101010...` and `01010101...`, we can compress it down to 1 bit, but the algorithm you wrote here would be useless. (Typically we'd compresss bytes rather than bits anyway, and this can easily be generalised to larger bases, so for this agorithm to be effective we'd use a base in which the data is independent.)

Another problem with encoding/compresion is that one may arbitrarily move information between the symbols and the language itself. For example, if we knew beforehand that in this notebook we would only compress these 3 pieces of data, we could have used 2 bits to store each of these three outcomes in our language. Our compressor would be useless practically, but would do very well on our tests. This kind of thing makes it hard to create a "compression test" using pre-determined data. This motivates _Kolmogorov complexity_, which measures the complexity of data not by its entropy, but by the amount of data required to generate it (e.g. the length of pseudo-code, or actual code, that creates it). How much, for example, could you compress a Mandelbrot set image?